In [0]:
import numpy as np
import math
import operator 
import random

class GeneticAlgorithm:
    
    def __init__(self ,fitness ,length ,size= 10 , selection_method = 'roulette',
                mutation_method = 'inversion',crossover_method = 'k-point',
                generation_method = 'random'):
        
        self.individuals_length = length
        self.fitness = fitness
        self.initialization = self.Initialization(generation_method,self.individuals_length)
        self.population = self.initialization.Generate(size)
        self.selection = self.Selection(fitness,selection_method)
        self.mutation = self.Mutation(mutation_method)
        self.crossover = self.CrossOver(crossover_method)
        
        
    class Selection :
        def __init__(self,fitness,method):
            
            self.method = method
            self.fitness = fitness
            
        def Select(self,population):
            
            return {'roulette': lambda: self.Roulette(population),
                    'stochastic-universal': lambda: self.StochasticUniversal(population),
                    'tournoment': lambda: self.Tournoment(population),
                    'truncate': lambda: self.Truncation(population)
                   }[self.method]()
        
        def Roulette(self,population):
            
            population_fitness = [self.fitness(individual) for individual in population ]
            total_fitness = sum(population_fitness)
            mean_fitness = total_fitness / len(population)
            
            new_population = []
            
            for n in range(len(population)):
                alpha = random.uniform(0, 1) * total_fitness
                totali = 0; j = 0 
                while True :
                    totali = totali + population_fitness[j]
                    j = j + 1
                    if not(j < n and totali < alpha) :
                        break
                new_population.append(population[j])

            return new_population
        
        def StochasticUniversal(self,population):
            population_fitness = [self.fitness(individual) for individual in population ]
            mean_fitness =np.mean(population_fitness)

            alpha = random.uniform(0, 1)
            total = population_fitness[0]
            delta = alpha * mean_fitness
            j = 0
            new_population = []
            
            while True :
                if delta < total :
                    new_population.append(population[j])
                    delta = total + delta
                else :
                    j = j + 1
                    if not (j < len(population)) :
                        break 
                    total = total + population_fitness[j]
                    
                
                    
            return new_population
        
        
        def Tournoment(self,population):
            
            k = int(random.uniform(0, 1)*len(population))
            
            t = population
            np.random.shuffle(t)
            t_fitness = [self.fitness(individual) for individual in t ]
            new_population = []
            
            for i in range(len(population)):
                j = 0
                while j < len(population):
                    i1 = j
                    for m in range(len(population)):
                        i2 = (j+m) % len(t)
                        if t_fitness[i1] > t_fitness[i2] :
                            new_population.append(t[i1])
                        else :
                            new_population.append(t[i2])
                    j = j + k 
                       
            return new_population
        
        def Truncation(self,population):
            
            alpha = random.uniform(0, 1)
            p_f = {p:self.fitness(p) for p in population}
            p_f = sorted(p_f.items(), key = lambda x : x[1])
            sp = int(alpha * len(population))
            j = 0
            new_population = []
            for item in p_f:
                if j > sp :
                    new_population.append(item[0])
                j = j + 1
                
            return new_population
    
    class Mutation:
        
        def __init__(self, method):
            self.method = method
            
        def Mutate(self,chromosome):
            
            return {'inversion': lambda: self.Inversion(chromosome)
                   ,'scramble': lambda: self.Scramble(chromosome)
                   ,'reverse': lambda: self.Reverse(chromosome)
                   ,'swap': lambda: self.Swap(chromosome)
                   }[self.method]()
        
        def Inversion(self,chromosome):
            
            while True :
                l = random.randint(0, len(chromosome))
                h = random.randint(0, len(chromosome))
                if h != l :
                    break
                    
            if l > h :
                l , h = h , l 
                
            mutated_chr = list(chromosome)
            
            for i in range(l,h):
                if chromosome[i]=='0':
                    mutated_chr[i] = '1'
                else :
                    mutated_chr[i] = '0'
                    
            return (''.join(mutated_chr))
        
        
        def Scramble(self, chromosome):
            
            l = list(chromosome)
            np.random.shuffle(l)
            mutated_chr = ''.join(l)
            
            return mutated_chr
            
            
        def Reverse(self,chromosome):
            
            while True :
                l = random.randint(0, len(chromosome)-1)
                h = random.randint(0, len(chromosome)-1)
                if h != l :
                    break
                    
            if l > h :
                l , h = h , l 
                
            mutated_chr = list(chromosome)
            
            for i in range(l,h):
                if chromosome[i]=='0':
                    mutated_chr[h-i] = '0'
                else :
                    mutated_chr[h-i] = '1'
                    
            return ''.join(mutated_chr)
        
        def Swap(self, chromosome):
            
            while True :
                l = random.randint(0, len(chromosome)-1)
                h = random.randint(0, len(chromosome)-1)
                if h != l :
                    break
                    
            if l > h :
                l , h = h , l 
                
            mutated_chr = list(chromosome)
            mutated_chr[h],mutated_chr[l] = chromosome[l],chromosome[h]
            
            return ''.join(mutated_chr)
        
        
    class CrossOver:
        
        def __init__(self,method):
            
            self.method = method
            
        
        def CrossOver(self,mother,father,k=1,proba=0.5):
            
            return {'k-point': lambda: self.K_Point(father,mother,k)
                   ,'uniform': lambda: self.Uniform(father,mother,proba)
                   }[self.method]()

        def K_Point(self, mother, father,k ):
            
            child_1 = mother
            child_2 = father
            ks = {}
            
            while True:
                point = int(random.uniform(0, 1) * len(mother))
                ks[point]=1
                child_1 = child_1[:point]+child_2[point:]
                child_1 = child_2[:point]+child_1[point:]
                if len(ks)==k:
                    break   
                    
            return child_1,child_2
         
        def Uniform(self, mother, father,proba):
            
            child_1 = list(mother)
            child_2 = list(father)
            
            for i in range(len(mother)):
                if random.random() > proba :
                    child_1[i],child_2[i] = child_2[i],child_1[i]
                
            return ''.join(child_1),''.join(child_2)
        
        
        
    def OffSpring(self,parents_population):
        
        random.shuffle(parents_population)
        i = 0
        j = len(parents_population)-1
        childs_population = []
        
        while i < j :
            child_1, child_2 = self.crossover.CrossOver(parents_population[i],parents_population[j])
            childs_population.append(child_1)
            childs_population.append(child_2)
            i = i + 1
            j = j - 1
        
        return childs_population
    
    def Optimization(self,iterations = 100,mutation_count = 1):
        
        population_tmp = self.population
        
        for i in range(iterations):
            parents_population = self.selection.Select(population_tmp)
            if len(parents_population) % 2 == 1:
                k = int(random.uniform(0, 1) * (len(parents_population)-1))
                del parents_population[k]
            population_tmp = [x for x in population_tmp if x not in parents_population]
            
            population_child = self.OffSpring(parents_population)
            if len(population_child):
                for i in range(mutation_count):
                    m = int(random.uniform(0, 1) * (len(population_child)-1))
                    population_child[m] = self.mutation.Mutate(population_child[m])
                    
                population_tmp += population_child
            
            
        fitness_values = {i : self.fitness(i) for i in population_tmp}
        
        return max(fitness_values.items(), key=operator.itemgetter(1))
                
        
    class Initialization:
        
        def __init__(self, method,length):
            self.length = length
            self.method = method
            
        def Generate(self,size):
            
            return {
                'random': lambda: self.Random(size),
                'n-queens': lambda: self.n_queens(size)
                }[self.method]()
        
        
        def Random(self,size):
            population = []
            
            while len(population) < size :
                r = random.randint(2**(self.length-1),2**(self.length ) - 1)
                population.append(str(bin(r))[2:])
                
            return population
        
        def n_queens(self,size):
            
            n = int(math.sqrt(self.length))
            population = []
            
            while len(population)<size:
                s = ''
                l = []
                for i in range(n):
                    l.append(random.randint(0,n-1))
                
                for i in range(n):
                    for j in range(n):
                        if j == l[i]:
                            s+='1'
                        else :
                            s+='0'
                        
                population.append(s)
            return population

In [0]:
def fitness(chromosome):
    return 1 * int(chromosome[0]) + 2 * int(chromosome[1]) + 4 * int(chromosome[2]) + 8 * int(chromosome[3]) - 16 * int(chromosome[4])


In [0]:
for sel in ['roulette','stochastic-universal']:
    for mut in ['inversion','scramble','reverse','swap']:
        for cr in ['k-point' ,'uniform']:
            print (sel + ' ' + mut  + ' '+ cr  + ' '+ ':') 
            ga = GeneticAlgorithm(fitness = fitness, length = 5, size = 3)
            print(ga.Optimization(iterations = 20, mutation_count = 3))

roulette inversion k-point :
('11110', 15)
roulette inversion uniform :
('11110', 15)
roulette scramble k-point :
('11110', 15)
roulette scramble uniform :
('11110', 15)
roulette reverse k-point :
('01110', 14)
roulette reverse uniform :
('11110', 15)
roulette swap k-point :
('00110', 12)
roulette swap uniform :
('00110', 12)
stochastic-universal inversion k-point :
('11110', 15)
stochastic-universal inversion uniform :
('01110', 14)
stochastic-universal scramble k-point :
('11110', 15)
stochastic-universal scramble uniform :
('11110', 15)
stochastic-universal reverse k-point :
('11110', 15)
stochastic-universal reverse uniform :
('11110', 15)
stochastic-universal swap k-point :
('00010', 8)
stochastic-universal swap uniform :
('01110', 14)


In [0]:
class nQueens:
    def __init__(self,n,size = 10):
        self.n = n
        self.size = size
        self.population = self.Generate()
        self.fitness = self.fitness
        
    def Generate(self):
        population = []
        for i in range(self.size):
            s = ''
            for j in range(self.n):
                r = random.randint(0,self.n-1)
                s+=str(r)
            population.append(s)
        return population
    
    
    
    def Select(self,population):
        population_fitness = [self.fitness(individual) for individual in population ]
        total_fitness = sum(population_fitness)
        mean_fitness = total_fitness / len(population)
            
        new_population = []
            
        for n in range(len(population)):
            alpha = random.uniform(0, 1) * total_fitness
            totali = 0; j = 0 
            while True :
                totali = totali + population_fitness[j]
                j = j + 1
                if not(j < n and totali < alpha) :
                    break
            new_population.append(population[j])

        return new_population
    def Mutate(self,chromosome):
        while True :
            l = random.randint(0, len(chromosome)-1)
            h = random.randint(0, len(chromosome)-1)
            if h != l :
                break
                    
        if l > h :
            l , h = h , l 
                
        mutated_chr = list(chromosome)
        mutated_chr[h],mutated_chr[l] = chromosome[l],chromosome[h]
            
        return ''.join(mutated_chr)
    
    
    def Crossover(self,mother,father):
        child_1 = list(mother)
        child_2 = list(father)
        proba = random.uniform(0,1)
        for i in range(len(mother)):
            if random.random() > proba :
                child_1[i],child_2[i] = child_2[i],child_1[i]
                
        return ''.join(child_1),''.join(child_2)
    
    
    def Offspring(self,parents_population):
        random.shuffle(parents_population)
        i = 0
        j = len(parents_population)-1
        childs_population = []
        
        while i < j :
            child_1, child_2 = self.Crossover(parents_population[i],parents_population[j])
            childs_population.append(child_1)
            childs_population.append(child_2)
            i = i + 1
            j = j - 1
        
        return childs_population
    
    
    def Optimization(self):
        population_tmp = self.population
        mutation_count = int(self.n / 2)
        fitness_values = {i : self.fitness(i) for i in population_tmp}

        while max(fitness_values.items(), key=operator.itemgetter(1)) != 28:
            parents_population = self.Select(population_tmp)
            if len(parents_population) % 2 == 1:
                k = int(random.uniform(0, 1) * (len(parents_population)-1))
                del parents_population[k]
            population_tmp = [x for x in population_tmp if x not in parents_population]
            
            population_child = self.Offspring(parents_population)
            if len(population_child):
                for i in range(mutation_count):
                    m = int(random.uniform(0, 1) * (len(population_child)-1))
                    population_child[m] = self.Mutate(population_child[m])
                    
                population_tmp += population_child
            
            
            fitness_values = {i : self.fitness(i) for i in population_tmp}
            
            
        for i in range(n):
            for j in range(n):
                if max(fitness_values.items(), key=operator.itemgetter(1))[0][j]==j:
                    print('1',end=' ')
                else:
                    print('0', end = ' ')
            print('\n')
        
        return max(fitness_values.items(), key=operator.itemgetter(1))
                
    def fitness(self,chromosome):
        clashes = 0;
            
        row_col_clashes = abs(len(chromosome) - len(np.unique(chromosome)))
        clashes += row_col_clashes

        for i in range(len(chromosome)):
            for j in range(len(chromosome)):
                if ( i != j):
                    dx = abs(i-j)
                    dy = abs(int(chromosome[i]) - int(chromosome[j]))
                    if(dx == dy):
                        clashes += 1


        return 28 - clashes
        

In [0]:
n_queen = nQueens(n=4,size = 10)

In [0]:
n_queen.Optimization()